In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
import time
import pymysql

# 화면에서 데이터 추출 함수
def get_page_info():
    for box in info_boxes:
        data = {}
        header = box.find('h4')
        data["회사명"] = header.find('span').text.strip()
        data["차종"] = header.find('p').text.strip()

        # dd 태그 안의 정보 추출
        for dd in box.find_all('dd'):
            label = dd.find('span').text.strip().replace('-', '').replace(':', '').strip()
            value = dd.text.replace(dd.find('span').text, '').strip().strip(':')
            data[label] = value

        car_list.append(data)

# 현재 페이지 번호 확인하기
def get_current_page():
    try:
        return driver.find_element(By.CSS_SELECTOR, 'a.current').text.strip()
    except:
        return None

# 세부차종 라디오버튼 정보 확인 및 다음 라디오버튼으로 이동
def chg_radio_button():
    radio_buttons = driver.find_elements(By.NAME, "evCarTypeDtl")

    # 현재 선택된 인덱스 찾기
    current_index = -1
    for i, radio in enumerate(radio_buttons):
        if radio.is_selected():
            current_index = i
            break

    # 다음 라디오 버튼이 있는 경우
    if current_index + 1 < len(radio_buttons):
        next_radio = radio_buttons[current_index + 1]
        next_radio_id = next_radio.get_attribute("id")

        # 해당하는 label 요소 찾기
        next_label = driver.find_element(By.CSS_SELECTOR, f"label[for='{next_radio_id}']")

        # 스크롤해서 보이게 하고 label 클릭
        driver.execute_script("arguments[0].scrollIntoView(true);", next_label)
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"label[for='{next_radio_id}']")))
        next_label.click()

        # 조회 버튼 클릭
        # search_button = driver.find_element(By.CSS_SELECTOR, '#searchForm > div > table > tbody > tr:nth-child(5) > td > button')
        # search_button.click()

        return True
    else:
        return False


# 크롤링 시작

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get("https://ev.or.kr/nportal/buySupprt/initSubsidyTargetVehicleAction.do")

car_list = []
prev_page = None

while True:

    time.sleep(2)
    # 화면에 데이터 있는지 판단
    info_boxes = BeautifulSoup(driver.page_source, 'html.parser').find_all('div', class_='infoBox')
    if not info_boxes: # 화면에 데이터가 없는 경우
        select = Select(driver.find_element(By.ID, "schCompany"))
        current_index = select.options.index(select.first_selected_option)
        next_index = current_index + 1

        if next_index < len(select.options): # selectbox 다음 옵션 선택
            select.select_by_index(next_index)
            driver.find_element(By.CSS_SELECTOR, '#searchForm > div > table > tbody > tr:nth-child(5) > td > button').click()
        else:
            #print("마지막 옵션이므로 다음 값이 없습니다.")
            more_modle_yn = chg_radio_button() # 라디오버튼 더 있는지 확인 / 없으면 while 종료
            if more_modle_yn == False:
                break

    else: # 화면에 데이터가 있는 경우
        get_page_info() # 데이터 정보를 가져와서 값을 append

        # 현재 페이지 번호 가져오기
        try:
            current_page = int(get_current_page())
        except:
            print("현재 페이지 정보를 가져오지 못함 → 종료")
            break

        # 현재 페이지 번호 저장
        prev_page = int(get_current_page())

        # 다음 페이지 버튼 클릭
        try:
            next_btn = driver.find_element(By.CSS_SELECTOR, '#pageingPosition a.next.arrow')
            next_btn.click()
            time.sleep(2)  # 클릭 후 로딩 대기
        except:
            print("다음 버튼 클릭 실패 → 라디오 버튼 이동 시도")
            more_model_yn = chg_radio_button()
            if not more_model_yn:
                break
            continue  # 조건 바꿨으니 다음 반복으로

        # 다음 페이지 번호 가져오기
        try:
            new_page = int(get_current_page())
        except:
            print("페이지 정보 확인 실패 → 종료")
            break

        # 만약 페이지가 이전과 같다면 → 조건 변경
        if new_page == prev_page:
            #print("페이지 동일 → select 또는 라디오 조건 변경 시도")

            # selectbox 변경 시도
            select = Select(driver.find_element(By.ID, "schCompany"))
            current_index = select.options.index(select.first_selected_option)
            next_index = current_index + 1

            if next_index < len(select.options):
                select.select_by_index(next_index)
                driver.find_element(By.CSS_SELECTOR, '#searchForm button').click()
            else:
                more_model_yn = chg_radio_button()
                if not more_model_yn:
                    break

# 크롤링 종료

# 데이터 DB 저장 시작
db = pymysql.connect(host='192.168.0.22', user='team_3', passwd='123', database='sk15_3team', port=3306)
cursor = db.cursor()

insert_query = """
INSERT INTO electric_cars (
    company_name, car_model, seat_capacity, max_speed, 
    range_per_charge, battery, subsidy, dealer_contact, 
    manufacturer, manufacturing_country
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

for car in car_list:
    cursor.execute(insert_query, (
        car.get('회사명'),
        car.get('차종'),
        car.get('승차인원'),
        car.get('최고속도출력'),
        car.get('1회충전주행거리'),
        car.get('배터리'),
        car.get('국고보조금'),
        car.get('판매사연락처'),
        car.get('제조사'),
        car.get('제조국가')
    ))

db.commit()
db.close()
# 데이터 DB 저장 종료